In [1]:
import websockets
import json
from datetime import datetime, timezone
import sys
import os
import asyncio

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from config import AIS_API_KEY, AIS_STREAM_URL, AIS_BOUNDING_BOXES


In [ ]:
async def connect_ais_stream():
    # Dictionary to store ship names (ship_id -> name)
    ship_names = {}
    
    async with websockets.connect(AIS_STREAM_URL) as websocket:
        subscribe_message = {
            "APIKey": AIS_API_KEY, 
            "BoundingBoxes": AIS_BOUNDING_BOXES
        }
        
        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)
        
        message_count = 0
        
        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            
            # Handle ShipStaticData to get ship names
            if message_type == "ShipStaticData":
                static_data = message.get('Message', {}).get('ShipStaticData', {})
                ship_id = static_data.get('UserID')
                ship_name = static_data.get('Name', '').strip()
                if ship_id and ship_name:
                    # Check if this is a new name or update
                    was_unknown = ship_id not in ship_names
                    ship_names[ship_id] = ship_name
                    if was_unknown:
                        # Log when we get a new ship name
                        print(f"Got name for ShipID {ship_id}: {ship_name}")
            
            if message_type == "PositionReport":
                # the message parameter contains a key of the message type which contains the message itself
                ais_message = message['Message']['PositionReport']
                message_count += 1
                
                # Extract data
                ship_id = ais_message.get('UserID', 'N/A')
                lat = ais_message.get('Latitude', 0)
                lon = ais_message.get('Longitude', 0)
                speed = ais_message.get('SpeedOverGround', None)
                course = ais_message.get('CourseOverGround', None)
                heading = ais_message.get('Heading', None)
                
                # Get ship name from cache
                ship_name = ship_names.get(ship_id, 'Unknown')
                
                # Format time
                time_str = datetime.now(timezone.utc).strftime("%H:%M:%S")
                
                # Build output string
                info_parts = []
                if speed is not None:
                    info_parts.append(f"Speed: {speed:.1f} kn")
                if course is not None:
                    info_parts.append(f"Course: {course:.1f}°")
                if heading is not None:
                    info_parts.append(f"Heading: {heading:.1f}°")
                
                info_str = " | ".join(info_parts) if info_parts else ""
                
                # Print formatted output with ship name
                print(f"[{time_str}] #{message_count:4d} | ShipID: {ship_id:12d} | "
                      f"Name: {ship_name:20s} | "
                      f"Lat: {lat:8.5f}° | Lon: {lon:9.5f}°" + 
                      (f" | {info_str}" if info_str else ""))

await connect_ais_stream()

[14:25:38] #   1 | ShipID:    567888886 | Name: Unknown              | Lat: 13.67256° | Lon: 100.58066°
Got name for ShipID 533132827: MTT LANGKAWI
[14:25:38] #   2 | ShipID:    563781000 | Name: Unknown              | Lat:  1.26224° | Lon: 103.81841°
[14:25:38] #   3 | ShipID:    564681000 | Name: Unknown              | Lat:  1.20168° | Lon: 103.74105°
[14:25:39] #   4 | ShipID:    563066530 | Name: Unknown              | Lat:  1.25125° | Lon: 103.78122°
[14:25:39] #   5 | ShipID:    244128000 | Name: Unknown              | Lat:  1.26492° | Lon: 103.86052°
Got name for ShipID 525100619: KT.SUBALI.2
[14:25:39] #   6 | ShipID:    563033100 | Name: Unknown              | Lat:  1.26581° | Lon: 103.85957°
[14:25:39] #   7 | ShipID:    636013579 | Name: Unknown              | Lat:  1.22606° | Lon: 103.92113°
[14:25:39] #   8 | ShipID:    563603000 | Name: Unknown              | Lat:  1.20658° | Lon: 103.68247°
[14:25:39] #   9 | ShipID:    564778000 | Name: Unknown              | Lat:  1.25